# Protein Sequence Cluster Analysis: 
### **For Retrieval of cluster representative sequence and Generation of Consensus Sequences from the clustered sequence.**

Author: Aaryesh Deshpande

## Overview
This notebook explores various clustering methods and generates consensus sequences from clustered groups. The goal is to analyze sequences, group them effectively, and retrieve representative sequences for further simulations.

---

## Methods Implemented
### Clustering Algorithms:
- **K-Means Clustering**: Partitions data into `k` clusters by minimizing the variance within each cluster.
- **Hierarchical Clustering**: Builds nested clusters by either merging or splitting them successively (Agglomerative Clustering).
- **BIRCH Clustering**: Constructs a tree structure for clustering large datasets efficiently.
- **DBSCAN Clustering**: Density-based algorithm that groups points closely packed together and identifies noise points.
- **OPTICS Clustering**: Similar to DBSCAN but can identify clusters of varying densities.
- **HDBSCAN**: Hierarchical density-based clustering to find clusters with variable densities.
- **Gaussian Mixture Model**: Assumes data points are generated from a mixture of Gaussian distributions.

### Feature Extraction:
Feature vectors are created using a combined approach:
1. **Sequence K-Mers**: Extract subsequences of length `k` from protein or DNA sequences to represent sequence patterns.
2. **Pseudo Amino Acid Composition (PAAC)**: Encodes sequences based on physicochemical properties:
    - **Hydrophobicity**
    - **Hydrophilicity**
    - **Mass**

### Representative Sequence Retrieval
#### Most Representative Sequence:
- The sequence closest to the cluster medoid is identified as the most representative sequence.

#### Consensus Sequences:
- Consensus sequences are generated for each cluster by aligning all sequences within the group and selecting the most frequent residues at each position using multiple sequence alignment generated by CLUSTAL and consensus generated by biopython.
---

## Applications:
- Functional annotation of proteins
- Reducing dataset size for computational simulations like Molecular Dynamics simulation.
- Identifying key representative sequences for downstream analysis

---

### Usage Instructions:
- Replace the file paths in the code with your desired input and output paths.
- Look for comments within the code to locate the relevant sections for modifying the file paths.

For detailed implementations, refer to the sections with Python code and comments. Each clustering algorithm and its corresponding feature extraction methods are described step-by-step. Each step is commented for clarity.

We welcome contributions to improve this project! Whether you've found a bug, have suggestions for fixes, or would like to add new features, your input is greatly appreciated.

---

### Processing FASTA File: Renaming and Tidying Headers

This section of the code focuses on tidying up the headers in a given FASTA file by renaming them in a systematic and consistent format. The renaming process starts with an anchor sequence named `sample_0` (the first sequence) and continues sequentially as `sample_01`, `sample_02`, ..., `sample_XX`.

#### Key Steps:
1. **Read and Parse the FASTA File**:
   - Input the original FASTA file containing sequences with varied or inconsistent headers.
   
2. **Rename Headers**:
   - Assign new headers in the format `sample_XX`, where `XX` represents the sequence number.
   - Ensure the first sequence is labeled `sample_0`.

3. **Save the Processed FASTA File**:
   - Write the updated sequences with tidy headers into a new FASTA file for downstream analysis.


In [1]:
# Function to read FASTA formatted file and return sequences as a list
def read_fasta(file_path):
    with open(file_path, 'r') as file:
        fasta_data = file.read().split(">")[1:]
    sequences = {}
    for lines in fasta_data:
        header_key = f"Sample_{len(sequences)}"
        seq_value = lines.split("\n", 1)[1].replace("\n", "").strip()
        sequences[header_key] = seq_value
    return sequences
    
#Replace with your FASTA file path and output file path
# Define Fasta File Path
fasta_file_path = "path/to/your/fasta_file.fa" # Change this to your FASTA file path
output_merge_path = "merged_fasta.fa" # Change this to your desired output file path

# Read the FASTA file and store the sequences in a dictionary
reader_obj = read_fasta(fasta_file_path)
reader_obj_values = list(reader_obj.values())

# Write the reader_obj_values sequences to a new FASTA file
with open(output_merge_path, 'w') as output_file:
    for idx, seq in enumerate(reader_obj_values):
        output_file.write(f">Sample_{idx}\n{seq}\n")


### Module Imports

This cell contains all the necessary module imports for the script, including standard Python libraries, third-party libraries, and specialized packages for bioinformatics and clustering.

#### Requirements:
Ensure the following libraries are installed in your environment before running this script. The required libraries can be installed using the following commands:

- **Using Conda**:
  ```bash
  conda install numpy pandas matplotlib biopython scikit-learn hdbscan scipy
  ```
- **Using pip**:
    ```bash
    pip install numpy pandas matplotlib biopython scikit-learn hdbscan scipy
    ```

- You need to also have clustio installed on your system for running the CLUSTALW command. To install clustio, use the following command:
    ```bash
    sudo apt install clustio (for linux)
    ```
To install all of the dependencies of this environment to run the script, use the ```bash environment_cluster.yml``` file to setup and install dependencies to a conda environment

In [40]:
# Default python libraries
import os
import glob

# 3rd party libraries
# Standard python libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Biopython libraries
from Bio import SeqIO
from Bio import AlignIO
from Bio.Align.AlignInfo import SummaryInfo
from Bio.Align.Applications import ClustalOmegaCommandline 

# Clustering libraries
import hdbscan
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, Birch, AgglomerativeClustering, OPTICS
from sklearn.metrics import silhouette_score, pairwise_distances
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

# Warning messages suppressor
import warnings
warnings.filterwarnings("ignore")

### K-mer Feature Extraction

This section focuses on generating sequence-based features using **K-mer extraction**. K-mers are subsequences of length `k` that are derived from a given sequence. The length of each k-mer is defaulted to 3, you can change that below to your own value.


In [3]:
# K-mer feature extraction
def kmer_count(sequence, k=3):
    kmer_dict = {}
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i+k]
        kmer_dict[kmer] = kmer_dict.get(kmer, 0) + 1
    return kmer_dict

# Generate k-mer matrix
def generate_kmer_matrix(sequences, k=3):
    kmer_matrices = []
    for sequence in sequences:
        kmer_matrices.append(kmer_count(sequence, k))
    return kmer_matrices

### Pseudo Amino Acid Composition (PAAC) Feature Extraction

This section extracts feature vectors based on Pseudo Amino Acid Composition (PAAC). The method involves calculating theta values to encode protein sequences effectively, enabling clustering and functional annotation.

The implementation is adapted from the work of **Rakesh Busi**. The original repository for PAAC implementation in clustering can be found [here](https://github.com/RakeshBusi/Clustering.git).

#### Reference
If you use this implementation, please cite the following article:

**Busi, Rakesh, Machingal, Pranav, Hemachandra, Nandyala, & Balaji, Petety V.**  
*How suitable are clustering methods for functional annotation of proteins?*  
bioRxiv, 2024.  
Publisher: Cold Spring Harbor Laboratory.  
DOI: [10.1101/2024.12.26.630370](https://doi.org/10.1101/2024.12.26.630370)


In [4]:
# PAAC feature extraction
class PseudoAminoAcidComposition:
    def __init__(self, fasta_data, lambda_=30, w=0.05):
        self.lambda_ = lambda_
        self.w = w
        self.physicochemical_properties = {
            'hydrophobicity': {'A': 0.62, 'C': 0.29, 'D': -0.90, 'E': -0.74, 'F': 1.19, 'G': 0.48, 'H': -0.40, 'I': 1.38, 'K': -1.50, 'L': 1.06, 'M': 0.64, 'N': -0.78, 'P': 0.12, 'Q': -0.85, 'R': -2.53, 'S': -0.18, 'T': -0.05, 'V': 1.08, 'W': 0.81, 'Y': 0.26},
            'hydrophilicity': {'A': -0.5, 'C': -1.0, 'D': 3.0, 'E': 3.0, 'F': -2.5, 'G': 0.0, 'H': -0.5, 'I': -1.8, 'K': 3.0, 'L': -1.8, 'M': -1.3, 'N': 0.2, 'P': 0.0, 'Q': 0.2, 'R': 3.0, 'S': 0.3, 'T': -0.4, 'V': -1.5, 'W': -3.4, 'Y': -2.3},
            'mass': {'A': 15.0, 'C': 47.0, 'D': 59.0, 'E': 73.0, 'F': 91.0, 'G': 1.0, 'H': 82.0, 'I': 57.0, 'K': 73.0, 'L': 57.0, 'M': 75.0, 'N': 58.0, 'P': 42.0, 'Q': 72.0, 'R': 101.0, 'S': 31.0, 'T': 45.0, 'V': 43.0, 'W': 130.0, 'Y': 107.0}
        }
        self.amino_acids = {idx + 1: aa for idx, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
        self.collect_features(fasta_data)

    # Collect features from the given FASTA data
    def collect_features(self, fasta_data):
        features = []
        features.append(['#'] + list(self.amino_acids.values()) + [f'λ{i}' for i in range(1, self.lambda_ + 1)])
        for i in range(0, len(fasta_data), 2):
            accession = fasta_data[i].strip()
            sequence = fasta_data[i + 1].strip()
            if len(sequence) < self.lambda_:
                print(f"Sequence {accession} skipped due to insufficient length.")
                continue
            paac = self.calculate_paac(sequence)
            features.append([accession] + paac)
        self.df = pd.DataFrame(features[1:], columns=features[0])

    # Calculate pseudo amino acid composition
    def calculate_paac(self, sequence):
        lower_theta = self.calculate_lower_theta(sequence)
        denominator = 1 + (self.w * sum(lower_theta.values()))
        paac = []
        for i in range(1, 21 + self.lambda_):
            if i <= 20:
                numerator = sequence.count(self.amino_acids[i]) / len(sequence)
                paac.append(numerator / denominator)
            else:
                numerator = self.w * lower_theta[i - 20]
                paac.append(numerator / denominator)
        return paac

    # Calculate lower theta values
    def calculate_lower_theta(self, sequence):
        lower_theta = {}
        for i in range(1, self.lambda_ + 1):
            if len(sequence) <= i:
                lower_theta[i] = 0
            else:
                lower_theta[i] = (1 / (len(sequence) - i)) * self.calculate_upper_theta(sequence, i)
        return lower_theta

    # Calculate upper theta values
    def calculate_upper_theta(self, sequence, i):
        upper_theta = []
        for j in range(len(sequence) - i):
            diff = [
                (self.physicochemical_properties[prop][sequence[j]] -
                 self.physicochemical_properties[prop][sequence[j + i]]) ** 2
                for prop in self.physicochemical_properties
            ]
            upper_theta.append(sum(diff) / len(self.physicochemical_properties))
        return sum(upper_theta)

### Combining Feature Sets: K-mers and PAAC

This section combines **K-mer features** and **Pseudo Amino Acid Composition (PAAC)** features for enhanced feature representation. By leveraging both sequence patterns and physicochemical properties, this approach improves the quality of input features for clustering and classification tasks.

In [5]:
# Load sequences
def load_sequences(file_path):
    return [str(record.seq) for record in SeqIO.parse(file_path, "fasta")]

# Combine K-mer and PAAC features
def combine_features(sequences, k=3, lambda_=30, w=0.05):
    kmer_features = generate_kmer_matrix(sequences, k)
    
    fasta_data = []
    for idx, seq in enumerate(sequences):
        fasta_data.extend([f">seq_{idx}", seq])
    
    paac = PseudoAminoAcidComposition(fasta_data, lambda_, w)
    paac_features = paac.df.iloc[:, 1:].values.tolist()
    
    combined_features = []
    max_length = max(len(list(kmer.values()) + paac) for kmer, paac in zip(kmer_features, paac_features))
    
    for kmer, paac in zip(kmer_features, paac_features):
        feature = list(kmer.values()) + paac
        # Pad with zeros if necessary
        feature += [0] * (max_length - len(feature))
        combined_features.append(feature)
    return np.array(combined_features, dtype=float)

### Clustering Methods and Evaluation

This section implements multiple clustering algorithms to group sequences based on extracted features. Each clustering method is evaluated using the Silhouette Score to determine the quality of clustering.

#### Clustering Functions:
1. **`kmeans_clustering`**: Performs K-means clustering.
2. **`dbscan_clustering`**: Density-based clustering using DBSCAN.
3. **`hierarchical_clustering`**: Hierarchical clustering with Ward's linkage.
4. **`birch_clustering`**: Efficient clustering using BIRCH.
5. **`agglomerative_clustering`**: Agglomerative hierarchical clustering.
6. **`optics_clustering`**: Clustering with OPTICS for variable density clusters.
7. **`hdbscan_clustering`**: Hierarchical density-based clustering (HDBSCAN), optimized to return the top 10 clusters by size.

#### Additional Features:
- **Evaluation Function**: `evaluate_clustering` computes the Silhouette Score for each clustering result.
- **Dimensionality Reduction**: PCA is applied to reduce the feature dimensions, improving clustering performance.
- **Best Clustering Selection**: The method with the highest Silhouette Score is selected as the best result. Silhouette Coefficient (SC): Measures how similar an object is to its own cluster compared to other clusters.

In [6]:
# Clustering functions
def kmeans_clustering(features, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(features)
    return labels

def dbscan_clustering(features, eps, min_samples):
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(features)
    return labels

def hierarchical_clustering(features, n_clusters):
    linkage_matrix = linkage(features, method='ward')
    labels = fcluster(linkage_matrix, n_clusters, criterion='maxclust')
    return labels

def birch_clustering(features, n_clusters):
    birch = Birch(n_clusters=n_clusters)
    labels = birch.fit_predict(features)
    return labels

def agglomerative_clustering(features, n_clusters):
    agglomerative = AgglomerativeClustering(n_clusters=n_clusters)
    labels = agglomerative.fit_predict(features)
    return labels

def optics_clustering(features, min_samples, xi, min_cluster_size):
    optics = OPTICS(min_samples=min_samples, xi=xi, min_cluster_size=min_cluster_size)
    labels = optics.fit_predict(features)
    return labels

def hdbscan_clustering(features, min_cluster_size, min_samples):
    clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples)
    labels = clusterer.fit_predict(features)
    
    # Count the occurrences of each label, excluding noise points (-1)
    unique_labels, counts = np.unique(labels[labels != -1], return_counts=True)
    
    # Sort clusters by size in descending order
    sorted_clusters = sorted(zip(unique_labels, counts), key=lambda x: x[1], reverse=True)
    
    # Select the top 10 clusters (or all if there are fewer than 10)
    top_clusters = sorted_clusters[:min(10, len(sorted_clusters))]
    top_cluster_labels = [label for label, _ in top_clusters]
    
    # Create a new label array with only the top clusters
    new_labels = np.full_like(labels, -1)
    for new_label, (old_label, _) in enumerate(top_clusters):
        new_labels[labels == old_label] = new_label
    
    return new_labels

# Evaluation function
def evaluate_clustering(features, labels):
    if len(set(labels)) > 1 and -1 not in set(labels):
        return silhouette_score(features, labels)
    else:
        return -1  # Invalid clustering (single cluster or noise)

# Main clustering function
def cluster_sequences(sequences, k=3, lambda_=30, w=0.05, n_clusters=10):
    # Extract combined features
    features = combine_features(sequences, k, lambda_, w)
    
    # Normalize features
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features)
    
    # Dimensionality reduction
    pca = PCA(n_components=min(50, len(normalized_features[0])))
    reduced_features = pca.fit_transform(normalized_features)
    
    # Perform clustering
    kmeans_labels = kmeans_clustering(reduced_features, n_clusters)
    dbscan_labels = dbscan_clustering(reduced_features, eps=0.5, min_samples=5)
    hierarchical_labels = hierarchical_clustering(reduced_features, n_clusters)
    birch_labels = birch_clustering(reduced_features, n_clusters)
    agglomerative_labels = agglomerative_clustering(reduced_features, n_clusters)
    optics_labels = optics_clustering(reduced_features, min_samples=5, xi=0.05, min_cluster_size=int(0.05 * len(sequences)))
    hdbscan_labels = hdbscan_clustering(reduced_features, min_cluster_size=5, min_samples=5)
    
    # Evaluate clustering results
    kmeans_score = evaluate_clustering(reduced_features, kmeans_labels)
    dbscan_score = evaluate_clustering(reduced_features, dbscan_labels)
    hierarchical_score = evaluate_clustering(reduced_features, hierarchical_labels)
    birch_score = evaluate_clustering(reduced_features, birch_labels)
    agglomerative_score = evaluate_clustering(reduced_features, agglomerative_labels)
    optics_score = evaluate_clustering(reduced_features, optics_labels)
    hdbscan_score = evaluate_clustering(reduced_features, hdbscan_labels)
    
    print(f"K-means Silhouette Score: {kmeans_score:.3f}")
    print(f"DBSCAN Silhouette Score: {dbscan_score:.3f}")
    print(f"Hierarchical Silhouette Score: {hierarchical_score:.3f}")
    print(f"Birch Silhouette Score: {birch_score:.3f}")
    print(f"Agglomerative Silhouette Score: {agglomerative_score:.3f}")
    print(f"OPTICS Silhouette Score: {optics_score:.3f}")
    print(f"HDBSCAN Silhouette Score: {hdbscan_score:.3f}")
    
    # Return the best clustering result
    all_labels = [kmeans_labels, dbscan_labels, hierarchical_labels, birch_labels, agglomerative_labels, optics_labels, hdbscan_labels]
    best_labels = max(all_labels, key=lambda x: evaluate_clustering(reduced_features, x))
    return best_labels, reduced_features


### Visualization of Clustering Results

This section provides functions for visualizing clustering results and understanding the distribution of sequences across clusters using PCA.

In [ ]:
def plot_clusters(features, labels, title):
    # Use PCA to reduce features to 2 dimensions for plotting
    pca = PCA(n_components=2)
    pca_features = pca.fit_transform(features)

    unique_labels = np.unique(labels)
    n_clusters = len(unique_labels)
    colors = plt.cm.tab10(np.linspace(0, 1, n_clusters))

    plt.figure(figsize=(10, 8))
    for label, color in zip(unique_labels, colors):
        if label == -1:  # Noise in DBSCAN or HDBSCAN
            plt.scatter(
                pca_features[labels == label, 0],
                pca_features[labels == label, 1],
                c='gray',
                marker='x',
                label='Noise'
            )
        else:
            plt.scatter(
                pca_features[labels == label, 0],
                pca_features[labels == label, 1],
                c=[color],
                label=f'Cluster {label}'
            )

    plt.title(title)
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.legend(loc='best', bbox_to_anchor=(1.05, 1), title="Clusters")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

def plot_dendrogram(linkage_matrix, title):
    plt.figure(figsize=(10, 7))
    dendrogram(linkage_matrix, truncate_mode='lastp', p=30, show_leaf_counts=True, leaf_rotation=45, leaf_font_size=12)
    plt.title(title)
    plt.xlabel('Cluster Size')
    plt.ylabel('Distance')
    plt.show()

def visualize_all_clusters(features, labels_dict):
    for method, labels in labels_dict.items():
        plot_clusters(features, labels, f"{method} Clustering")

# Example usage in your code:

file_path = output_merge_path  # Replace with your FASTA file path in the first block
sequences = load_sequences(file_path)
best_labels, reduced_features = cluster_sequences(sequences)
# Generate labels_dict after clustering
labels_dict = {
    "K-means": kmeans_clustering(reduced_features, n_clusters=10),
    "Classical DBSCAN": dbscan_clustering(reduced_features, eps=0.5, min_samples=5),
    "Hierarchical": hierarchical_clustering(reduced_features, n_clusters=10),
    "Birch": birch_clustering(reduced_features, n_clusters=10),
    "Agglomerative": agglomerative_clustering(reduced_features, n_clusters=10),
    "OPTICS": optics_clustering(reduced_features, min_samples=5, xi=0.05, min_cluster_size=int(0.05 * len(sequences))),
    "HDBSCAN": hdbscan_clustering(reduced_features, min_cluster_size=5, min_samples=5),
}
plot_dendrogram(linkage(reduced_features, method='ward'), "Hierarchical Clustering Dendrogram")
# Visualize all clustering results
visualize_all_clusters(reduced_features, labels_dict)

#### Saving Clustered Sequences by Method and Selecting the Best Clustering Method for Consensus Sequences

- All cluster files are saved in separate directories based on the clustering method used. Each file contains the sequences grouped by their respective clusters.
- The best-performing clustering method (often K-means, as it works well in most cases) is used as an example in the following analysis blocks to generate sequence consensus and representation.
- This ensures that the most accurate and representative sequences are captured for downstream analysis.

In [ ]:
# Save clustered sequences to files
def save_cluster_sequences(sequences, labels_dict, output_dir, original_headers):
    for method_name, labels in labels_dict.items():
        # Create method-specific directory
        method_dir = os.path.join(output_dir, method_name)
        os.makedirs(method_dir, exist_ok=True)
        
        # Group sequences by cluster
        clusters = {}
        for seq, label, header in zip(sequences, labels, original_headers):
            if label not in clusters:
                clusters[label] = []
            clusters[label].append((seq, header))
        
        # Save sequences for each cluster
        for label, cluster_seqs in clusters.items():
            cluster_file = os.path.join(method_dir, f"cluster_{label}.fasta")
            with open(cluster_file, 'w') as f:
                for seq, header in cluster_seqs:
                    f.write(f"{header}\n{seq}\n")
        
        print(f"Saved {len(clusters)} clusters for {method_name} in {method_dir}")

# Use the function with the labels dictionary
labels_dict = {
    "K-means": kmeans_clustering(reduced_features, n_clusters=10),
    "Classical DBSCAN": dbscan_clustering(reduced_features, eps=0.5, min_samples=5),
    "Hierarchical": hierarchical_clustering(reduced_features, n_clusters=10),
    "Birch": birch_clustering(reduced_features, n_clusters=10),
    "Agglomerative": agglomerative_clustering(reduced_features, n_clusters=10),
    "OPTICS": optics_clustering(reduced_features, min_samples=5, xi=0.05, min_cluster_size=int(0.05 * len(sequences))),
    "HDBSCAN": hdbscan_clustering(reduced_features, min_cluster_size=5, min_samples=5)
}

# Assuming you have a list of original headers
original_headers = [f">Sample_{i}" for i in range(len(sequences))]

# Make sure to replace "output_clusters" with your desired output directory path.
save_cluster_sequences(sequences, labels_dict, "output_clusters", original_headers) 

## Representative Sequence Retrieval:
### **1. Finding the Medoid of Clusters for the Best Represented Sequence (Minimal Center)**

This section focuses on identifying the **medoid** of clusters to select the best-represented sequence within each cluster. The **K-Means** clustering method is chosen due to its effectiveness in grouping sequences and its ability to minimize outlier noise.

#### Key Concepts:
- **Centroid**:
  - Represents the geometric center of a cluster.
  - Calculated as the average of all data points within the cluster.
  - Note: The centroid is not guaranteed to be an actual data point.

- **Medoid**:
  - A data point within the cluster that is most centrally located.
  - Defined as the point that minimizes the sum of distances to all other points in the cluster.
  - Considered the best-represented sequence as it is an actual member of the cluster.

#### Why Medoid over Centroid?
- While centroids are useful for understanding the general location of a cluster, they do not necessarily correspond to real sequences.
- Medoids, being actual data points, are more representative and interpretable for downstream analysis like simulations or functional annotations. This approach ensures the selection of sequences that are not only central to their clusters but also real and relevant for analysis.

In [23]:
# Compute the medoid
def compute_medoid(cluster_data):
    distances = pairwise_distances(cluster_data)  # Compute pairwise distances
    medoid_index = np.argmin(distances.sum(axis=0))  # Find the point with the smallest total distance
    return medoid_index  # Return the index of the medoid

# Function to compute medoids and map back to original data
def medoid_seq_with_data(features, labels, sequences):
    unique_labels = np.unique(labels)
    medoids = {}
    medoid_sequences = {}
    
    for label in unique_labels:
        if label == -1:  # Skip noise points - as they don't have a cluster
            continue
        # Get the indices of points in the current cluster
        cluster_indices = np.where(labels == label)[0]
        cluster_points = features[cluster_indices]
        
        # Get the medoid index within the cluster
        medoid_index_within_cluster = compute_medoid(cluster_points)
        medoid_index = cluster_indices[medoid_index_within_cluster]  # Map back to the original index
        
        # Save the medoid feature and corresponding sequence
        medoids[label] = features[medoid_index]
        medoid_sequences[label] = sequences[medoid_index]
    
    return medoids, medoid_sequences

# Example with K-means labels (best in most cases), you can replace it with any other clustering method's labels by changing the dict index.
medoids, medoid_sequences = medoid_seq_with_data(reduced_features, labels_dict["K-means"], sequences) 

# Print medoid sequences
print("\nMedoid Sequences:")
for cluster_id, medoid_sequence in medoid_sequences.items():
    print(f"Cluster {cluster_id}: {medoid_sequence}")
    
# Save medoid sequences to a file in a new folder called representative_seq_mediods
# Replace with your desired output directory
os.makedirs("representative_sequences", exist_ok=True)
with open("representative_sequences/medoid_sequences.fasta", 'w') as f:
    for cluster_id, medoid_sequence in medoid_sequences.items():
        f.write(f">{cluster_id}\n{medoid_sequence}\n")
print("\nMedoid sequences saved to representative_sequences/medoid_sequences.fasta")

### **2. Deriving the Optimal Consensus Sequence to Represent Each Cluster**

- **Multiple Sequence Alignment (MSA):**  
  Perform multiple sequence alignment for all sequences within each cluster using a robust MSA tool. This step ensures accurate alignment and highlights conserved residues critical to the biological function or structural integrity of the sequences.

- **Consensus Sequence Generation:**  
  Based on the aligned sequences, compute the consensus sequence that best represents the cluster. The consensus sequence reflects the most frequent or conserved residues at each position, providing a generalized representation of the cluster's functional and structural properties.


#### **CLUSTAL Alignment for Clustered Sequences**

- **Sequence Alignment with CLUSTAL**:  
  Use the CLUSTAL tool to perform multiple sequence alignment (MSA) on sequences within each cluster. This ensures accurate alignment of residues, facilitating the identification of conserved regions and patterns.

- **Output Format**:  
  Save the alignment results in a `.aln` file format, which is a standard format for representing aligned sequences and is compatible with various bioinformatics tools for further analysis.

- **Purpose**:  
  - Highlight conserved regions within the cluster to identify key functional or structural motifs.  
  - Generate high-quality alignment data for consensus sequence derivation or downstream analysis.

**Note:**
- Ensure the sequences are cleaned and formatted properly (e.g., consistent headers) before alignment.
- We use Biopython's CLUSTALIO wrapper for python to perform our analysis. You can use other alignment algorithms like MUSCLE. 


In [ ]:
def clustal_cluster(input_dir, output_dir, clustalo_path="clustalo"):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Find all FASTA files in the input directory
    fasta_files = glob.glob(os.path.join(input_dir, "*.fasta"))
    
    if not fasta_files:
        print("No FASTA files found in the input directory.")
        return
    
    # Iterate over each FASTA file and perform alignment
    for fasta_file in fasta_files:
        cluster_name = os.path.splitext(os.path.basename(fasta_file))[0]
        output_file = os.path.join(output_dir, f"{cluster_name}_aligned.aln")
        
        # Create the Clustal Omega command
        clustalomega_cline = ClustalOmegaCommandline(
            cmd=clustalo_path,
            infile=fasta_file,
            outfile=output_file,
            outfmt = 'clustal',  
            verbose=True,
            auto=True
        )
        
        # Print and run the command
        print(f"Running Clustal Omega for {fasta_file}...")
        try:
            stdout, stderr = clustalomega_cline()
            print(f"Alignment completed for {fasta_file}. Output saved to {output_file}")
        except Exception as e:
            print(f"Error processing {fasta_file}: {e}")

# Define the input and output directories
input_dir = "output_clusters/K-means"  # Path to the directory containing K-means clustered FASTA files - U can change what clustering you want to use
output_dir = "consensus_sequences/alignments"    # Directory to save the aligned cluster sequences

# Run the Clustal Omega alignment
clustal_cluster(input_dir, output_dir)

#### **Generating Consensus Sequences**

- **Consensus Sequence Derivation**:  
  Utilize Biopython's `SummaryInfo` module to generate a consensus sequence from the aligned sequences. The consensus sequence represents the most frequent residue at each position, ensuring it reflects the key features of the aligned cluster.

- **Threshold Setting**:  
  Apply a consensus threshold of **0.7**, meaning that a residue must appear in at least 70% of the sequences at a given position to be included in the consensus. Positions with lower agreement will be represented with an ambiguous character (e.g., "X" or "N") in our case as "N". Changes can be made to the threshold and ambiguous character according to the needs.



In [ ]:

def consensus_sequences(msa_dir, output_file):
    # Find all MSA files (aligned FASTA) in the directory
    msa_files = glob.glob(os.path.join(msa_dir, "*.aln"))
    
    if not msa_files:
        print("No MSA files found in the input directory.")
        return
    
    # Open the output file for writing consensus sequences
    with open(output_file, "w") as output_handle:
        for msa_file in msa_files:
            try:
                # Read the MSA file
                alignment = AlignIO.read(msa_file, "clustal")
                
                # Check if alignment contains sequences
                if not alignment:
                    print(f"Skipping empty or invalid file: {msa_file}")
                    continue
                
                # Compute the consensus sequence
                summary_info = SummaryInfo(alignment)
                consensus = summary_info.dumb_consensus(threshold=0.7, ambiguous="N") # Adjust threshold and ambiguous character as needed
                
                # Extract the cluster name from the file name
                cluster_name = os.path.splitext(os.path.basename(msa_file))[0]
                
                # Write the consensus sequence to the output file
                output_handle.write(f">{cluster_name}_consensus\n{consensus}\n")
                print(f"Consensus sequence generated for {msa_file} and saved.")
            
            except Exception as e:
                print(f"Error processing {msa_file}: {e}")
    
    print(f"All consensus sequences saved to {output_file}.")
    
msa_dir = "consensus_sequences/alignments"  # Directory containing the aligned cluster FASTA files

# Replace with your desired output file path
output_file = "consensus_sequences/cluster_consensus_sequences.fasta"  # Path to save the consensus sequences

consensus_sequences(msa_dir, output_file)